# Evaluating Machine Learning Models

Machine learning is awesome, right?! Define "awesome".

The goal of machine learning algorithms is to **extract meaningful patterns** from training data either to help us understand that training data or to use those patterns to **make inferences** about new data (or both). If our goal is to better understand our training data, how do we know that the identified patterns are reliable insights into our data? If our goal is to make inferences about new data, how do we estimate the performance of our model on the new data? To succesfully apply machine learning methodology in research or industry applications, we must be able to answer these questions. While tools like IBM Watson greatly simplify the actual training of Machine Learning models, the careful evaluation of those models by the practicioner is indispensible.

In this notebook, we will 

    1. Define what we mean by "awesome"
    2. Introduce some common pitfalls on the road to awesome
    3. Discuss how to calculate awesomeness
    4. Leverage this knowledge to create even awesomer models
    
While this notebook include code examples, the content is intended for a general audience. The concepts that we discuss here have relevance for almost any data analysis project and should help you think critically and strategically whether you are a coder or not! If you do want to play with the code, feel free! You can double-click a cell to edit it. Press Shift+Enter to execute the cell. If you want to save your progress, you will first have to save a copy of the notebook to your drive. 

## Evaluating _Supervised_ Models
In supervised learning, we fit a model that predicts a target variable $y$ given som input features $X$ by looking at lots of pre-labelled example pairs, $(y_1, X_1), (y_2, X_2), \ldots, (y_N, X_N)$. These pre-labelled examples make the evaluation of supervised models relatively straightforward compared to unsupervised.

For the remainder of this notebook, we will use an example from a political science project where we are modeling whether Tweets are political attacks based on the text of the tweet. The data set below includes our target column called `attack` and many feature columns called `tweet_vec_n` where n ranges from 1 to 75. In the attack column, 1s correspond to attacks and 0s correspond to non-attacks. The labels were assigned by humans. The `tweet_vec_n` columns form a numerical representation of the text of the tweet. Very similar tweets should have similar numbers. If this sounds crazy, don't worry about it for now as you won't need to understand this to understand the concepts in this notebook.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t

In [2]:
data = pd.read_csv('/zfs/citi/workshop_data/python_ml/attacks_with_vecs.csv')
display(data)
print("\nShape of full data (num. rows, num. cols.): ", data.shape)

,attack,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,...,vec_66,vec_67,vec_68,vec_69,vec_70,vec_71,vec_72,vec_73,vec_74,vec_75
0,0,0.004863,0.016942,0.024855,0.033888,-0.034978,-0.031188,-0.005101,-0.049382,0.035185,...,0.034461,-0.010897,-0.041929,-0.000717,0.057181,0.017889,-0.030877,-0.039927,0.027440,0.016859
1,1,0.026694,0.010135,-0.018828,0.046631,-0.022001,0.010430,0.009476,-0.039059,-0.007851,...,-0.004756,-0.012542,0.028331,0.005426,0.015014,0.010646,-0.029356,-0.027508,0.053418,0.024528
2,0,-0.080366,-0.015909,-0.014785,-0.022373,0.113360,-0.029767,0.008176,0.003181,-0.059703,...,0.013075,-0.006471,0.018220,-0.027452,0.028064,-0.020817,-0.107500,0.024037,0.009529,-0.033408
3,0,0.015414,-0.013312,-0.021079,0.045578,0.040534,0.018513,0.052609,-0.016076,0.052258,...,0.043686,-0.029191,0.027954,-0.037915,0.036626,0.005471,-0.028403,-0.089455,0.070966,-0.020188
4,1,0.029376,0.027177,-0.041911,0.020376,-0.005441,-0.019955,-0.005843,-0.006628,-0.006893,...,0.015489,-0.014160,0.024181,-0.007469,0.010908,0.012564,-0.017050,-0.012459,0.057483,0.035029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,0,0.000710,-0.021085,-0.021229,0.049246,-0.010870,0.007334,0.001050,-0.027241,0.004095,...,0.032286,-0.024165,0.009352,-0.002891,0.027083,0.022846,-0.060983,-0.016991,0.011163,-0.038895
1271,1,0.007446,0.012536,-0.092788,-0.009357,-0.000470,-0.018887,0.014990,0.024066,0.056112,...,0.023681,-0.017766,0.033713,-0.015553,-0.051125,0.045200,0.023976,-0.013475,0.037990,-0.001396
1272,0,0.025122,-0.058875,0.035881,0.038083,-0.060735,-0.019663,0.019662,-0.002776,-0.047708,...,-0.010515,0.057939,-0.031716,0.007308,0.015016,0.121070,-0.051429,-0.004486,0.021345,-0.061438
1273,0,-0.038971,-0.035440,0.049050,-0.028902,0.085988,0.039999,0.007620,-0.068482,0.035770,...,-0.052571,-0.033085,0.032628,0.016246,0.022087,-0.014952,0.045145,-0.014866,0.051695,0.008636



Shape of full data (num. rows, num. cols.):  (1275, 76)


For use throughout, we will split this data into
* $y$ - our "target" indicating whether or not the tweet is an attack. 
* $X$ - the length 75 numerical vectors representing the tweet text.

We wish to find a model $f$ such that
$$
y\approx f(X).
$$

In [3]:
# data = data[data.sample_id==2]
y = data.attack
X = data.iloc[:,1:]

print("Shape of y: ", y.shape)
print("Shape of X: ", X.shape)

Shape of y:  (1275,)
Shape of X:  (1275, 75)


### Basic Concepts
These lay the conceptual groundwork for understanding how to evaluate machine learning models (or statistical models in general). 

#### Don't test your model on the training data!
![train_test_split](https://miro.medium.com/max/2272/1*-8_kogvwmL1H6ooN1A1tsQ.png)
The number one rule for evaluating machine learning models is, **don't test your model on the same data you used to train the model!** Doing so is kindof like reusing homework problems verbatim on an algebra exam. The algebra students could get a perfect score on the exam simply by memorizing the answers to each homework problem without learning anything about algebra. This is unsatisfactory, because if faced with a new problem the algebra student would be completely lost. Similarly, under some conditions, ML algorithms are prone to "memorizing" responses rather than learning generalizeable patterns. Even in circumstances where the model is not precisely memorizing the data, it will perform at least somewhat better on the training set than on a held-out test set, so one should never expect the performance on the training set to generalize to new data.

To demonstrate this idea, we will split our data $y$, $X$ into a training set $y_\mathrm{train}$, $X_\mathrm{train}$ and a test set $y_\mathrm{test}$, $X_\mathrm{test}$. We then train a model on $y_\mathrm{train}$ and look at the mean accuracy when we test our model on the training set and the test set. By the way, the type of model we use here is called [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

In [4]:
# compute polynomial features up to degree 2
from sklearn.preprocessing import PolynomialFeatures
X_poly = PolynomialFeatures(2).fit_transform(X)
X_poly.shape

(1275, 2926)

In [5]:
# make the train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.333, random_state=42)
print("X_train.shape=", X_train.shape)
print("X_test.shape=", X_test.shape)

X_train.shape= (850, 2926)
X_test.shape= (425, 2926)


In [6]:
# fit the model on the training data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=10000, solver='liblinear')
model.fit(X_train, y_train);

In [7]:
def fmt_pct(rat):
    return str(round(rat*100, 1)) + "%"

print("Mean accuracy on the training set: " + fmt_pct(model.score(X_train, y_train)))
print("Mean accuracy on the test set: " + fmt_pct(model.score(X_test, y_test)))

Mean accuracy on the training set: 99.8%
Mean accuracy on the test set: 88.7%


Training set accuracy is much higher than test set accuracy!

### Rigorous Model Evaluation with Cross Validation

#### shortcoming of simple train/test split
The train/test split has the latent assumption that the error on the test set is a good proxy for the error on new unseen data. But our test sample will always be non-representative, to some extent, of the total population. In the train/test approach, you don't have any way to estimate how different your test data is from the population.  

#### k-Fold Cross Validation

We've emphasized the importance of using a witheld test set to evaluate our models in order to get an unbiased estimate of model performance. But what if our test set happens to be substantially different from the full population that we are sampling from? It may be that we substantially underestimate or worse overestimate the performance of our model. For instance, maybe some Tweets are very easy to classify as attack or non-attack while others are very difficult to classify with a given modeling approach. If we happen to get a larger portion of the easy cases in our test dataset, then our evaluation metrics (precision, recall, F1, ROC-AUC) will be larger than if we had a representative distribution of easier and harder Tweets! This problem is especially pronounced with smaller datasets such as ours. Note that this is not an error of bias, but rather a random statistical error. 

To demonstrate, let's split our dataset into several groups of equal size and withold in turn each of the five as our test dataset. We will train the model on the remaining four, and calculate the ROC-AUC score on the fifth to see how much the score varies from group to group. The figure below demonstrates this procedure:

![cross validation](https://i.stack.imgur.com/1fXzJ.png)

The code below performs this splitting and outputs the AUC score for each of 10 splits:

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# define how splitting is performed
cv = StratifiedKFold(n_splits=5)
model = LogisticRegression(C=10000, solver='liblinear')

# loop over splits
i=1
aucs = []
for train, test in cv.split(X, y):
    # train model
    model_split = model.fit(X.iloc[train], y.iloc[train])
    # get test probs
    y_test_probs = model_split.predict_proba(X.iloc[test])[:,1]
    # get roc-auc
    auc_split = roc_auc_score(y.iloc[test], y_test_probs)
    aucs.append(auc_split)

    print("Split " + str(i) +". ROC-AUC: " , fmt_rat(auc_split, 3))
    i=i+1

NameError: name 'fmt_rat' is not defined

#### Overfitting and underfitting

The problem we encountered above results from a phenomenon called **overfitting**. During the fitting process, the model learns (or memorizes) many of the specifics of the training data. This memorization doesn't apply to new data, so we see a big difference in the performance on the training data and the test data. This difference might be OK as long as the performance on the test set is acceptable. Sometimes to learn generalizeable patterns from the training data, you have to settle for a bit of overfitting. However, too much overfitting will eventually hurt performance. This source of error is sometimes called model _variance_. We'll make this idea more concrete later. 

**Underfitting** is the opposite of overfitting and occurs when the model is not sufficiently flexible to capture the general trends in the data. This source of error is sometimes called model _bias_.

The figure below summarizes both of these ideas. We would like to arrive at a model (blue line) which captures the overall behavior of the data (orange dots) and that will generalize well to new data sampled from the same distribution. The straight line model on the left is insufficiently flexible to capture the curved shape of the data and is said to be _underfitting_. The wiggly line model on the right is so flexible that it's fitting random noise in the data rather than the overall behavior and is said to be _overfitting_. The middle model does a good job of capturing the overall behavior of the data and is said to be _juuuuuust right_.

![Model under and over-fitting](https://docs.aws.amazon.com/machine-learning/latest/dg/images/mlconcepts_image5.png)

#### The bias-variance tradeoff

Above, we discussed how both overfitting and underfitting can harm the performance of our model. So we should somehow try to minimize them. However, reducing the error resulting from one tends to increase the error resulting from the other. This phenomenon is called the bias-variance tradeoff, where bias and variance are understood to be sources of error resulting from underfitting and overfitting respectively. In particular:
  * **Bias error** results from a model that is improperly constrained (or biased) in a way that prevents it from representing the actual patterns in the data, such as the left-most plot above. 
  * **Variance error** results from a model that is so unconstrained that it fits the noise in the sample dataset preventing it from generalizing well to new data such as the rightmost plot above. The term variance refers to how, since the model is fitting the noise of the sample data, the parameters of the model would change dramatically if trained on a new sample.
 
A good machine learning model finds the sweet spot which balances these two sources of error. But how exactly do we do that? There are many ways, and we won't get into the specific now. However, to demonstrate the idea, we will take our attack data and train progressively more complex models and see how the error changes on the training set and the test set. To acheive this, we again use a Logistic Regression but with progressively larger values for a parameter called the ["inverse regularization strength"](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) or "C" for short. The basic idea is that larger values of C result in more flexible models that have more potential for overfitting; small values of C result in less flexible models leading to bias. We will train models for values of C between 0 and 1,000 and plot the result.

In [ ]:
# we will return to GridSearchCV later in the notebook
from sklearn.model_selection import GridSearchCV

# We want to see how the model performs 
params = {"C": np.logspace(-1,4,20)}

# train the model at the specified max depth
model = LogisticRegression(solver='liblinear', max_iter=200)
model_cv = GridSearchCV(model, params, cv=15, 
                        scoring='roc_auc', 
                        n_jobs=6,
                        refit=True,
                        return_train_score=True,
                        verbose=10)

model_cv.fit(X,y);

bias_variance = params['C']
scores_train = model_cv.cv_results_["mean_train_score"]
scores_test = model_cv.cv_results_["mean_test_score"]

print("Optimal value. C=" + str(round(model_cv.best_estimator_.C)))

# Plot the scores
plt.semilogx(bias_variance, scores_train, 'ro-', label='train')
plt.semilogx(bias_variance, scores_test, 'bo-', label='test')
plt.legend()
plt.xlabel("(<--Bias) C (Variance-->)")
plt.ylabel("Model Performance")
plt.title("Bias-Variance Tradeoff")
plt.show();

---

**Question**: Why does the training set keep going up, while the test set goes up then comes back down?  

---

#### The class imbalance problem
Often with classification problems, some classes will be more abundant than others. This leads to a few issues which are collectively called the **class imbalance problem**:
  1. The model will tend to perform much better for the classes that have more data
  2. Model metrics like accuracy will tend to give an inflated sense of the model performance.
  
Let's look at how many attacks and non-attacks we have in our dataset:

In [ ]:
data.attack.value_counts() / len(data)

We see that only about 13% of our tweets are attacks. Let's imagine a very simple model where we just say that every tweet is not an attack. This stupid model knows nothing about attacks, and yet, due to the portion of non-attacks in our dataset,  it is 87% accurate! 

Look back at the Bias-Variance tradeoff plot above. On the far left, the model accuracy approaches 88%. At the time we may have thought this was pretty good. Now we see that the stupid model is simply predicting that none of the tweets are attacks.

Clearly we need new metrics which take class imbalance into account.

A few important things to observe from this plot:
* On the far left, the training and test accuracies are similar. These models have little overfitting (good), but the error due to bias makes the accuracy quite low compared to the rest of the plot (bad). 
* As we move toward the middle of the plot, we see the train accuracy rise far above the test accuracy, corresponding to the onset of model variance error. However, the test accuracy is continueing to rise, so that's OK. 
* Once we get beyond $C=100$ the test accuracy starts to drop even though the train accuracy continues to rise due to large model variance error. The peak at $C\sim50$ is the optimal balance between the two sources of error! 

#### Confidence scores and decision thresholds
Most classification models output a confidence score rather than a discreet label. For example, our attack classifier will output a number between 0 and 1. The closer this score is to 1, the more "confident" the model is that the tweet is an attack. Let's look at the confidence scores for our model. 

The figures below show the confidence scores that our model predicted on held-out test data. The figure on the left shows the confidence scores for tweets that we know are actually non-attacks. As we would expect, the confidence scores are mostly close to zero. The figure on the right shows the scores for tweets that we know to be attacks. Compared to the non-attack tweets, these scores are shifted toward 1 as we would hope. There are quite a few attack tweets that have very low confidence scores. It would be interesting to look at these to see why the model is failing to grasp that they are attacks.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
clf = model_cv.best_estimator_.fit(X_train, y_train)

X_test_attack = X_test[y_test==1]
X_test_non_attack = X_test[y_test==0]

confidence_scores_attack = clf.predict_proba(X_test_attack)[:,1]
confidence_scores_non_attack = clf.predict_proba(X_test_non_attack)[:,1]

ax=plt.subplot(1,2,1)
plt.hist(confidence_scores_non_attack,bins=10)
plt.vlines(0.5,0,500, colors='k', linestyles='dotted')
plt.xticks([0,0.25,0.5,0.75,1.0])
plt.ylim(0,500)
plt.title("Actual Non-attacks")
plt.text(0.25, 0.99,'Predicted Non-attacks\n(True Negatives)',
     horizontalalignment='center',
     verticalalignment='top',
     transform = ax.transAxes)
plt.text(0.75, 0.99,'Predicted Attacks\n(False Positives) ',
     horizontalalignment='center',
     verticalalignment='top',
     transform = ax.transAxes)
plt.xlabel("Confidence score")
plt.ylabel("Number of tweets")

ax=plt.subplot(1,2,2)
plt.hist(confidence_scores_attack,bins=10)
plt.vlines(0.5,0,300, colors='k', linestyles='dotted')
plt.xticks([0,0.25,0.5,0.75,1.0])
plt.ylim(0,35)
plt.title("Actual Attacks")
plt.text(0.25, 0.99,'Predicted Non-attacks\n(False Negatives)',
     horizontalalignment='center',
     verticalalignment='top',
     transform = ax.transAxes)
plt.text(0.75, 0.99,'Predicted Attacks\n(True Positives) ',
     horizontalalignment='center',
     verticalalignment='top',
     transform = ax.transAxes)
plt.xlabel("Confidence score")

fig = plt.gcf()
fig.set_size_inches(10, 5)
plt.show()

---

**Question**: Why do you think the model appears to be struggling more in the case of attacks than non-attacks? 

---

In many cases, we will need to turn the confidence score between 0 and 1 into a model prediction, non-attack or attack. How do we do this? 

As suggested by the plot above, we can define a decision threshold (vertical dotted line) so that everything to the left of the vertical line is predicted to be a non-attack and everything to the left is predicted to be an attack. In the plots, the decision threshold is set at 0.5, but there's no rule that says we have to set it there. Depending upon our model and application, we may want to choose a smaller or larger value. Note that when we calculated accuracy in previous examples, we assumed a decision threshold of 0.5. 

We'll discuss decision thresholds in more detail when we get to the classification metrics section below.

## Evaluation Metrics

### Metrics for regression models

##### MSE/RMSE, MAE, and R2

Mean Square Error (MSE) is one of the most commonly used model metrics for regression models -- where we're trying to use our model to predict some quantity. 

The MSE model metric is simple. To evaluate a model's MSE, you simply use the model to predict each target in your test set. For each target you predicted, you get some error -- how far your prediction is from the target. If you take all your errors, square them, and sum them, you have the MSE of your model on the test set. To get the RMSE, you just take the square root of the MSE.

MSE and RMSE are essentially two ways of expressing the same information. At root, RMSE is more interpretable because it is on the same scale as the original data -- if your target is, e.g.,  *dollars*, then the RMSE will also be expressed in dollars. On the other hand, MSE can be more practical, because it's slightly easier to work with (mathematically) in most contexts.

Let's take a look at an application of MSE.

The below code fits a simple linear regression model for a single measure of diabetes progression. The regressor in this case is body mass index (BMI).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Load the diabetes dataset
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

# Use only one feature
diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes_y[:-20]
diabetes_y_test = diabetes_y[-20:]

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(diabetes_X_train, diabetes_y_train)

# Make predictions using the testing set
diabetes_y_pred = regr.predict(diabetes_X_test)

# The coefficients
print('Coefficients: ', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(diabetes_y_test, diabetes_y_pred))
print('Root mean squared error: %.2f'
      % np.sqrt(mean_squared_error(diabetes_y_test, diabetes_y_pred)))
print('Mean absolute error: %.2f'
      % mean_absolute_error(diabetes_y_test, diabetes_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(diabetes_y_test, diabetes_y_pred))

# Plot outputs
plt.scatter(diabetes_X_test, diabetes_y_test,  color='black')
plt.plot(diabetes_X_test, diabetes_y_pred, color='blue', linewidth=3)

plt.xlabel('BMI (centered, standardized)')
plt.ylabel('Measure of diabetes progression')

plt.show()

That's all well and good, but our dataset actually contains more than just BMI; it also contains information about patients' age and blood pressure. We could use a multiple linear regression model that includes *all* of these measures, but it's also interesting to see which of them is the best one to include in a simple (single-regressor) model. Among other things, this helps us figure out which measure is most closely related to diabetes.

Toward that end, let's look at our model metrics for each of the three models under consideration; the one (already examined above) based on age, one based on BMI, and one based on blood pressure.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Load the diabetes dataset
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

# Use only one feature
diabetes_X_age = diabetes_X[:, np.newaxis, 0]
diabetes_X_bp = diabetes_X[:, np.newaxis, 3]

# Split the data into training/testing sets
diabetes_X_age_train = diabetes_X_age[:-20]
diabetes_X_age_test = diabetes_X_age[-20:]
diabetes_X_bp_train = diabetes_X_bp[:-20]
diabetes_X_bp_test = diabetes_X_bp[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes_y[:-20]
diabetes_y_test = diabetes_y[-20:]

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(diabetes_X_age_train, diabetes_y_train)

# Make predictions using the testing set
diabetes_y_pred = regr.predict(diabetes_X_age_test)

# The coefficients
print('AGE:')
print('Coefficients: ', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(diabetes_y_test, diabetes_y_pred))
print('Root mean squared error: %.2f'
      % np.sqrt(mean_squared_error(diabetes_y_test, diabetes_y_pred)))
print('Mean absolute error: %.2f'
      % mean_absolute_error(diabetes_y_test, diabetes_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(diabetes_y_test, diabetes_y_pred))

# Plot outputs
plt.scatter(diabetes_X_age_test, diabetes_y_test,  color='black')
plt.plot(diabetes_X_age_test, diabetes_y_pred, color='blue', linewidth=3)

plt.xlabel('Age (centered, standardized)')
plt.ylabel('Measure of diabetes progression')

plt.show()

print('\n\n')

# Train the model using the training sets
regr.fit(diabetes_X_bp_train, diabetes_y_train)

# Make predictions using the testing set
diabetes_y_pred = regr.predict(diabetes_X_bp_test)

# The coefficients
print('BLOOD PRESSURE:')
print('Coefficients: ', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(diabetes_y_test, diabetes_y_pred))
print('Root mean squared error: %.2f'
      % np.sqrt(mean_squared_error(diabetes_y_test, diabetes_y_pred)))
print('Mean absolute error: %.2f'
      % mean_absolute_error(diabetes_y_test, diabetes_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(diabetes_y_test, diabetes_y_pred))

# Plot outputs
plt.scatter(diabetes_X_bp_test, diabetes_y_test,  color='black')
plt.plot(diabetes_X_bp_test, diabetes_y_pred, color='blue', linewidth=3)

plt.xlabel('Blood pressure (centered, standardized)')
plt.ylabel('Measure of diabetes progression')

plt.show()

We see that age is not nearly as good a basis for inferring diabetes progression as is BMI. Blood pressure is much closer -- much better than age -- but is still not as good as BMI. So if, for some reason, we wanted to choose a single metric to use to predict someone's progression in diabetes measures, we would settle on BMI.

### Metrics for classification models

##### TP, FP, TN, FN and the Confusion Matrix

An understanding of True Positives (TP), False Positives (FP), True Negatives (TN), and False Negatives (FN) is the foundation for understanding all of the other metrics we will discuss in the context of classification. This topic is not complicated, but it can be a challenge to keep everything straight in your head. 

First, we need to understand what we mean by _Positive_ and _Negative_. Both of these refer to one of the classes in a two-class comparison, and actually it's up to you to decide which class is called "Positive" and which class is called "Negative". Usually, the more interesting class is assigned the Positive label. For our attack tweet data, this would be the tweets which are attacks (non-attacks are Negatives). In a cancer screening application, this would be the cases where the subject is cancerous (non-cancerous are Negatives). Usually, but not always, the positive class ends up being the minority class. 

Now we're ready to understand TP, FP, etc.:
* **True Positive**:  _predicts_ positive; _actually_ positive
* **True Negative**:  _predicts_ negative; _actually_ negative
* **False Positive**:  _predicts_ positive; _actually_ negative
* **False Negative**:  _predicts_ negative; _actually_ positive

The matrix below, called the **Confusion Matrix**, summarizes the performance of a model by looking at the number of test set examples that fall into each possible scenario. This is a very common way to present model performance, so it's worth pausing to make sure you understand it. 

|   |Actually Negative   | Actually Positive  |
|:-:|:-:|:-:|
|  **Predicted Negative** |  True Negative | False Negative  |
|  **Predicted Positive** |  False Positive | True Positive  |

Above, we discussed how most machine learning models output a confidence score, and, in order to get class label predictions, we have to apply a decision threshold. Clearly, then, the values in the confusion matrix will depend on our choice for the decision threshold. Look again at the [confidence scores histograms](https://colab.research.google.com/drive/1jw_JRCrGougID6n2mruf2BqksImGrlB-?authuser=1#scrollTo=FkJsS_hWUzt7) above. Notice how the different regions of the plot are lablled. It's worth stopping to make sure these labels make sense to you.

Below we show the confusion matrix for the best model that we trained in the [bias-variance tradeoff](https://colab.research.google.com/drive/1jw_JRCrGougID6n2mruf2BqksImGrlB-?authuser=1#scrollTo=TzFhGIhX6WJL) section (looking at the plot in that section, this is the model with C=100). The 3 different confusion matrices below are for three different values of the decision threshold (0.25, 0.5, and 0.75). 

In [ ]:
from sklearn.metrics import confusion_matrix
y_prob = model_cv.predict_proba(X_test)[:,1]

# Note that sklearn uses the opposite convention for rows and columns than used above
# So, we transpose the output of the sklearn function.

# Confusion Matrix (eps = 0.25)
cm25 =  confusion_matrix(y_test, (y_prob>0.25).astype(int)).transpose()
print("Confusion matrix for threshold=0.25: \n", cm25)

# Confusion Matrix (eps = 0.50)
cm50 = confusion_matrix(y_test, (y_prob>0.50).astype(int)).transpose()
print("Confusion matrix for threshold=0.50: \n", cm50)

# Confusion Matrix (eps = 0.75)
cm75 = confusion_matrix(y_test, (y_prob>0.75).astype(int)).transpose()
print("Confusion matrix for threshold=0.75: \n", cm75)

How would you describe what happens to outputs of our model as we increase the decision threshold?

In [ ]:
# save the results
df_results = pd.DataFrame([])
df_results['y_test'] = y_test
df_results['y_predicted_prob'] = y_prob
df_results.to_csv('logistic_results.csv', index=False)

##### TP, FP, TN, FN Rates

The numbers in the confusion matrix above scale with the size of our test dataset. This makes it hard to compare between different applications or test data sets. To solve this problem, we can define rates which scale with the size of the test dataset:
* **True Positive Rate (TPR)** -- TP divided by the number of actual positives, $P$ : 
$$ TPR = \frac{TP}{P} = \frac{TP}{TP+FN} $$
This metric answers the question, "what fraction of the positive cases will the model correctly identify?". You may see this called "sensitivity", "recall", or "hit rate" -- sorry.
* **True Negative Rate (TNR)** - TN divided by the number of actual negative, $N$:
$$ TNR = \frac{TN}{N} = \frac{TN}{TN+FP} $$
You may see this called "specificity". Because the positive class is usually more interesting, this metric is not used very frequently. 
* **False Positive Rate (FPR)** - FP divided by the number of actual negatives:
$$ FPR = \frac{FP}{N} = \frac{FP}{TN+FP} = 1-TNR$$
This is the fraction of negatives that were incorrectly predicted to be positive. False positives are often very costly (think of the cancer screening example). You may see this called the "fall-out rate". 
* **False Negative Rate (FNR)** - FN divided by the number of actual positives:
$$ FNR = \frac{FN}{P} = \frac{FN}{TP+FN} = 1-TPR$$
This is the fraction of positives that were incorrectly predicted to be negative. Like TNR, you won't run into this one very often. Sometimes this is called the "miss rate". 

Who can remember all these names?! TPR and FPR are the most important here. 

Let's look at TPR and FPR for our model for the three threshold values we used above:

In [ ]:
def cm_rates(cm):
    tpr = cm[1,1]/cm[:,1].sum()
    tnr = cm[0,0]/cm[:,0].sum()
    fpr = 1 - tnr
    fnr = 1 - tpr
    
    return dict(tpr= tpr, tnr= tnr, fpr= fpr, fnr= fnr)

cmr25 = cm_rates(cm25)
cmr50 = cm_rates(cm50)
cmr75 = cm_rates(cm75)

print("Threshold = 0.25:\nTPR: ", round(cmr25['tpr'],2), "\nFPR: ", round(cmr25['fpr'],2))
print("\nThreshold = 0.50:\nTPR: ", round(cmr50['tpr'],2), "\nFPR: ", round(cmr50['fpr'],2))
print("\nThreshold = 0.75:\nTPR: ", round(cmr75['tpr'],2), "\nFPR: ", round(cmr75['fpr'],2))

Why is there a tradeoff between TPR and FPR as we change the threshold?

How does class imbalance effect our interpretation of these numbers?

##### Accuracy

Accuracy answers the question, "what fraction of my model's predictions were correct?". 
$$
\mathrm{Accuracy} = \frac{TP + TN}{P + N} 
$$
Accuracy is very sensitive to class imbalance. 

##### Precision and Recall

Taken together, these two metrics give a good sense for the performance of a binary classification model:

* **Precision**: the percentage of predicted attacks that are actually attacks.  Higher precision suggests that when a model predicts that a tweet is an attack, it is more likely to be correct.  Note that a model with high precision can still be inadequate if the model fails to correctly identify a sufficient number of the actual attacks.  A precision of 1 can be achieved simply by predicting that no tweets are attacks. In terms of quantities defined above
$$ \mathrm{Precision} = \frac{TP}{\mathrm{Predicted\, Positives}} = \frac{TP}{TP + FP} $$
* **Recall**: the percentage of actual attacks that are predicted to be attacks.  Higher recall suggests that if a tweet is actually an attack, the model is more likely to identify it as such.  Note that a model with very high recall can still be inadequate if an insufficient number of the predicted attacks are actually attacks.  A recall of 1 can be achieved by simply predicting that all tweets are attacks. Recall is equal to $TPR$ defined above:
$$ \mathrm{Recall} = TPR = \frac{TP}{P} = \frac{TP}{TP+FN} $$

The figure below demonstrates these metrics visually. The "relevant" label is what we have been calling "Actual Positives". 

![precision and recall](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/350px-Precisionrecall.svg.png)

Like all of the metrics we've discussed so far, precision and recall depend upon the decision threshold. We can increase the precision by decreasing the threshold at the cost of decreasing the recall or visa-versa. The Precision-Recall curve, shown below, looks at the values of precision and recall as the threshold is tuned between 0 and 1. The numbered points show the decision threshold for a few points along the curve.


In [ ]:
def fmt_rat(flt,dec=3):
    flt = str(round(flt,dec))
    return flt

# Precision-Recall Analysis:
from sklearn import metrics
precision, recall, eps = metrics.precision_recall_curve(y_test, y_prob)

# precision recall curve
fig, ax = plt.subplots();
ax.plot(recall, precision, 'b-')
ax.set(xlabel='recall', ylabel='precision',ylim=(0,None))
ax.set_aspect('equal')
plt.title("Precision-Recall Curve")

# annotations
idx = np.linspace(0,len(eps)-3,20).astype(int)-1
ax.plot(recall[idx], precision[idx], 'ko')
for i in idx:
    ax.annotate(fmt_rat(eps[i],2), (recall[i]+.02, precision[i]+.02), 
                horizontalalignment='left', 
                verticalalignment='bottom',
                fontsize=12
               )
    
fig.set_size_inches(6,6)
plt.show()
plt.clf()

Note how, overall, precision and recall trade off with one another. Ideally, we would like both to be as high as possible, but we may be more concerned about one type of error. For the anaysis of attack tweets, we may be content with a lower recall (not picking out all attacks) as long as we have high precision (most predicted attacks are actually attacks). So, we could choose a threshold that favors precision over recall, such as the point near $\epsilon=0.86$ where the precision is near $90\%$ and the recall is near $25\%$. 

We pointed out earlier how *accuracy* can be misleading for imbalanced datasets. Is there a similar problem with *precision* and *recall*?

##### F1 Score


The **F1 score**: a composite metric equal to the harmonic mean of precision and recall:
$$
F1 = \frac{\mathrm{precision} \times \mathrm{recall}}{\tfrac{1}{2}\left(\mathrm{precision} + \mathrm{recall}\right)}
$$ 
By combining precision and recall in this way, $F1$ represents the overall performance for a specified threshold. Below, we show the maximum possible F1 for our trained model along with the associated precision, recall, and decision threshold:


In [ ]:
f1 = 2 * precision * recall / (precision + recall)
f1=f1[f1==f1]
f1max_ind = np.argmax(f1)
print("Maximum F1 score: " + fmt_rat(f1[f1max_ind]))
print("Precision at max: " + fmt_rat(precision[f1max_ind]))
print("Recall at max: " + fmt_rat(recall[f1max_ind]))
print("Optimal threshold: " + fmt_rat(eps[f1max_ind]))

Try to find this point on the plot above. When will this be a good choice for the decision threshold?

While F1 does give a single number for model performance, that number still depends upon the decision threshold which is based on application-specific considerations. It would be great to have a single metric which didn't depend on a choice of threshold!

Does F1 do a good job of measuring model performance for datasets with class imbalance? Why or why not?


##### Receiver Operating Characteristic (ROC)
The **Receiver Operating Characteristic (ROC)** Curve shows the true-positive rate vs the false-positive rate. Like the Precision-Recall curve, the points are generated by different choices of the decision threshold. Also like precision and recall, TPR and FPR trade off against one another as we vary the threshold. Below, we show the ROC Curve for our trained model evaluated on the test data:

In [ ]:
# ROC-AUC Analysis
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob, pos_label=1)
roc_auc_test = metrics.roc_auc_score(y_test, y_prob)

# roc curve
fig, ax = plt.subplots() 
fig.set_size_inches(6,6)
ax.plot(fpr, tpr, 'b-', label='Model, AUC: ' + fmt_rat(roc_auc_test))
ax.plot([0,1],[0,1],'k--', label='Chance, AUC: 0.5')
ax.set(xlabel='FPR', ylabel='TPR')
ax.set_aspect('equal')

# annotations
idx = np.linspace(0,len(thresholds)-3,15).astype(int)-1
ax.plot(fpr[idx], tpr[idx], 'ko')
for i in idx:
    ax.annotate(fmt_rat(thresholds[i],2), (fpr[i]+0.02, tpr[i]-.05), 
                horizontalalignment='left', 
                verticalalignment='bottom',
                fontsize=12
               )

ax.legend()
plt.title("ROC Curve")
plt.show()
plt.clf()


The area under the ROC curve (**ROC-AUC**) turns out to be a great threshold-independent way to quantify overall model performance. There are several nice properties:

1. The AUC is independent of choice of threshold! This means we don't have to make any assumptions about how the model will be used when evaluating the goodness of fit of the model.
2. A coin-flip model will produce an AUC of 0.5, and a perfect model will produce an AUC of 1. These boundaries make it easy to interpret the ROC score.
3. AUC only depends on the sorting of the confidence scores predicted by the model, not the actual confidence scores themselves. Well-fitting, but poorly calibrated models will still receive strong ROC scores.
4. Because of points 1 and 3, the ROC-AUC can be directly compared for different models of the same data.
5. The ROC-AUC score is not biased to be larger/smaller for class-imbalanced datasets.

ROC-AUC is a great metric to use while you are experimenting with different modeling approaches and trying to decide what works best!

#### shortcoming of simple train/test split
The train/test split has the latent assumption that the error on the test set is a good proxy for the error on new unseen data. But our test sample will always be non-representative, to some extent, of the total population. In the train/test approach, you don't have any way to estimate how different your test data is from the population.  

#### k-Fold Cross Validation

We've emphasized the importance of using a witheld test set to evaluate our models in order to get an unbiased estimate of model performance. But what if our test set happens to be substantially different from the full population that we are sampling from? It may be that we substantially underestimate or worse overestimate the performance of our model. For instance, maybe some Tweets are very easy to classify as attack or non-attack while others are very difficult to classify with a given modeling approach. If we happen to get a larger portion of the easy cases in our test dataset, then our evaluation metrics (precision, recall, F1, ROC-AUC) will be larger than if we had a representative distribution of easier and harder Tweets! This problem is especially pronounced with smaller datasets such as ours. Note that this is not an error of bias, but rather a random statistical error. 

To demonstrate, let's split our dataset into several groups of equal size and withold in turn each of the five as our test dataset. We will train the model on the remaining four, and calculate the ROC-AUC score on the fifth to see how much the score varies from group to group. The figure below demonstrates this procedure:

![cross validation](https://i.stack.imgur.com/1fXzJ.png)

The code below performs this splitting and outputs the AUC score for each of 10 splits:

In [ ]:
from sklearn.model_selection import StratifiedKFold

# define how splitting is performed
cv = StratifiedKFold(n_splits=5)

# loop over splits
i=1
aucs = []
for train, test in cv.split(X, y):
  # train model
  model_split = model_cv.best_estimator_.fit(X.iloc[train], y.iloc[train])
  # get test probs
  y_test_probs = model_split.predict_proba(X.iloc[test])[:,1]
  # get roc-auc
  auc_split = metrics.roc_auc_score(y.iloc[test], y_test_probs)
  aucs.append(auc_split)
  
  print("Split " + str(i) +". ROC-AUC: " , fmt_rat(auc_split, 3))
  i=i+1

The different splits give quite different scores! 

Often the AUC score is used to choose between candidate models. Given the variation we see above, how could we decide between competing models?  One obvious step is to use the mean value of the 10 scores we calculated above:

In [ ]:
mean_auc = np.mean(aucs)
print("Mean value of ROC-AUC scores for "+ str(len(aucs)) + " splits: ", fmt_rat(mean_auc))

This is better than just using one random test set, but it still doesn't take into account the variation among the auc scores. The standard error of the mean tells us how uncertain our estimate of this mean value is given the variation in the auc scores and the number of splits. The standard error of the mean (SEM) is estimated as 
$$
\mathrm{SEM} = \frac{\mathrm{Standard \, Deviation \, of \, Scores}}{\sqrt{\mathrm{Number \, of \, Scores}}}
$$
The code below calculates the standard error:

In [ ]:
SEM_auc = np.std(aucs) / np.sqrt(len(aucs))
print("SEM value of ROC-AUC scores for 10 splits: ", fmt_rat(SEM_auc))

So, even though the variation in auc score between splits is quite large, the error in our estimate of the mean is quite small. Statisticians love 95% confidence intervals. Assuming the auc scores are normally distributed, we can now calculate a 95% confidence interval in the estimated 95% confidence interval. It turns out that the bounds of this confidence interval equals
$$
\mathrm{95\% \, Conf.\, Int.} = \mathrm{Mean \, AUC  \mp t(\alpha=0.05, \mathrm{df}=9)\times SEM}
$$
The code below calculates this confidence interval:

In [ ]:
t_score = np.abs(t.ppf(0.05, 10-1))
print("t-value:", t_score)
print("95%% Conf. Int. in mean ROC-AUC score: [%0.3f, %0.3f]" % (mean_auc -  t_score * SEM_auc, mean_auc + t_score * SEM_auc))

In [ ]:
# put the above steps for computing confidence interval into a function
def conf_int(ar):
  mean = np.mean(ar)
  sem = np.std(ar) / np.sqrt(len(ar))
  t_score = np.abs(t.ppf(0.05, len(ar)-1))
  return (mean - t_score * sem, mean + t_score * sem )

Now we are ready to compare different models!

Say we used a completely different model, went through the procedure above and arrived at a confidence band $[0.942, 0.961]$. In that case we would be very confident that the latter model was superior. Alternatively, if the second model had a confidence band of $[0.920, 0.95]$ we wouldn't be sure -- the second model could be superior, but the differences could also be explained by the error resuling from the fact that our data set is just a sample.

It turns out that the procedure we went through above has a name -- **k-Fold Cross Validation**. This is the de-facto method for calculating robust estimates of model performance. Make sure you understand it!

Can you think of any limitations or pitfalls with this procedure? 

To demonstrate the real application of this procedure, we will repeat the process above, but we will also train a classification model called a Multilayer Perceptron Classificer in addition to the Logistic Regression model that we've been working with above. Multilayer Perceptrons are the simplest type of Neural Network.

In [ ]:
from sklearn.neural_network import MLPClassifier as Clf

mlp_params = {
    'alpha':  np.logspace(-0.8,0.3,8, 30, 80),
    'hidden_layer_sizes': [(100,),]
}

mlp_cv = GridSearchCV(Clf(max_iter=400), mlp_params, scoring='roc_auc', cv=4, n_jobs=6, verbose=10)
mlp_cv.fit(X,y)

In [ ]:
bias_variance = mlp_params['alpha']
scores_test = mlp_cv.cv_results_["mean_test_score"]

print("Optimal value: alpha=%0.3f" % mlp_cv.best_estimator_.alpha)

# Plot the scores
plt.plot(bias_variance, scores_test, 'bo-', label='test')
plt.legend()
plt.xlabel("(<--Variance) alpha (Bias-->)")
plt.ylabel("ROC_AUC")
plt.xscale('log')
plt.title("Bias-Variance Tradeoff")
plt.show();

In [ ]:
#@title
from sklearn.model_selection import StratifiedKFold
#from sklearn.svm import SVC as Clf

# define how splitting is performed
cv = StratifiedKFold(n_splits=10)

# loop over splits
i=1
aucs_mlp = []
for train, test in cv.split(X, y):
  Xtr, ytr = X.iloc[train], y.iloc[train]
  Xte, yte = X.iloc[test], y.iloc[test]
  
  # train models
  model_split = mlp_cv.best_estimator_.fit(Xtr, ytr)
  
  # get test probs
  y_test_probs = model_split.predict_proba(Xte)[:,1]
  # get roc-auc
  auc_split = metrics.roc_auc_score(yte, y_test_probs)
  aucs_mlp.append(auc_split)
  
  print("MLP Split " + str(i) +". ROC-AUC: " , fmt_rat(auc_split, 3))
  i=i+1

Now let's calculat the 95% confidence interval for the ROC-AUC and compare with the Logistic Regression model from before:

In [ ]:
#@title
print("Logistic Regression: \n  Mean ROC-AUC:", 
      fmt_rat(np.mean(aucs)),
      "\n  95% Conf. Int.:", 
      conf_int(aucs),
      "\n"
     )

print("Multilayer Perceptron Classifier: \n  Mean ROC-AUC:", 
      fmt_rat(np.mean(aucs_mlp)),
      "\n  95% Conf. Int.:", 
      conf_int(aucs_mlp)
     )



The Mean ROC-AUC for the Multilayer Perceptron is higher than Logistic Regression, and the confidence bands have almost no overlap. We can now say with confidence that the neural network model performs better with regard to ROC-AUC. 

#### Variants
There are many variants of cross validations that work better or worse in various situations. For example, for very small datasets, k-fold cross validation has the problem, that there isn't enough data to get enough sub-sample estimates to accuractly estimate the sample mean and SEM. In this case, a technique called Repeated k-fold cross validation can be useful. See here for the sklearn implementation: [https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedKFold.html](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedKFold.html). 

[This guide](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation), also from sklearn, gives a fantastic overview of various cross-validation techniques.

## How to prevent overfitting

So say you've got a model and you know or suspect that your model variance is too high, and you're overfitting your training data. What strategies can you take to fix this? There are three simple steps you can take, which we'll consider one by one.

#### Reduce model complexity

The first strategy is to reduce the complexity of your model. This is usually (but not always) the same as reducing the number of parameters in your model.

One way to think about this is that the parameters of the model -- for example, the coefficients of the regression terms in a linear regression model, or the weights and biases in a neural network -- are the way the model stores information. If your model has enough "storage capacity", then it will solve the AI problem you're working on by just "memorizing" the way your training data looks. 

Think of it like when instructors let you bring a 3x5 index card of notes to an exam. They specify 3x5 because if they let you bring giant broadsheets of notes, then your learning might be too shallow; you will pass the exam just by virtue of having brought all that information with you. Reducing you to a 3x5 requires you to do some deep learning before the exam. AI is the same way; **reducing the model complexity forces the model to learn the underlying phenomenon that produced the training data, instead of just learning the training data itself**.

Let's see this in action. Below, I generate some data for a regression problem. I have one feature, `X`, and I generate a response `Y` that we want to try to model. Since I'm the one making this data, I can tell you that `Y` is just three times `X` with some noise added. But don't tell the model that -- we want to see how well the model can learn this relationship just by looking at some training data.

In this cell, I generate the data, separate it into train and test sets, and plot the training data.

In [ ]:
# Make some fake data

# Import what we need
from sklearn.model_selection import train_test_split
from numpy.random import default_rng
rng = default_rng(355)

# Make the data
datasize=14
Xlin = np.sort(rng.uniform(low=-4,high=4,size=datasize))
Y = 3*Xlin + rng.normal(loc=1,scale=8,size=datasize)

# Divide into train and test
Xlin_train, Xlin_test, Y_train, Y_test = train_test_split(
    Xlin, Y, test_size=0.5, random_state=355)

plt.plot(Xlin_train,Y_train,'ro')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

Now let's try a model that is overly complex -- a 4-degree polynomial regression. The real phenomenon is a 1-degree polynomial: `Y = 3X + noise`. Of course, since we're pretending we don't know the true relationship between `X` and `Y`, we also wouldn't know that this model is too complex. But in general, high-degree polynomial fits are prone to being overly complex. It's very rare you'd want to go above a 3-degree polynomial.

In the cell below, we fit a degree 4 polynomial regression model on the training data and look at the resulting fit.

In [ ]:
# Fit complex model (high-degree polynomial regression)
high_degree=4
hdp = np.poly1d(np.polyfit(Xlin_train,Y_train,deg=high_degree))

# Take a look
Xplot = np.linspace(-4,4)
fig, axs = plt.subplots(1,2,figsize=(12,5))
axs[0].plot(Xplot,hdp(Xplot))
axs[0].plot(Xlin_train,Y_train,'ro')
axs[1].plot(Xplot,hdp(Xplot))
axs[1].plot(Xlin_train,Y_train,'ro')
axs[1].set_ylim(-25,15)
plt.show()

You can see that our line fits the training data quite well. You also might be worried about that enormous dropoff on the right, that doesn't seem to be reflected in our training data. This is one of the dangers of high-degree polynomial fits!

In the cell below we find the RMSE of the complex model on the training set as well as on the validation set. We also add the test set data to the plot.

In [ ]:
# Get RMSE on train and test set

# Import
from sklearn.metrics import mean_squared_error

# Get metrics
train_rmse = mean_squared_error(Y_train, hdp(Xlin_train), squared=False)
test_rmse = mean_squared_error(Y_test, hdp(Xlin_test), squared=False)

# Print
print('Training set RMSE: ',train_rmse)
print('Test set RMSE: ',test_rmse)

# Take a look
Xplot = np.linspace(-4,4)
fig, axs = plt.subplots(1,2,figsize=(12,5))
axs[0].plot(Xplot,hdp(Xplot))
axs[0].plot(Xlin_train,Y_train,'ro')
axs[0].plot(Xlin_test,Y_test,'bo')
axs[1].plot(Xplot,hdp(Xplot))
axs[1].plot(Xlin_train,Y_train,'ro')
axs[1].plot(Xlin_test,Y_test,'bo')
axs[1].set_ylim(-25,15)
plt.show()


Both the RMSE numbers and the plots look pretty abysmal. We've fit a model that is flexible enough to fit itself very nicely to the training data, and in the process it completely misses the true underlying phenomenon. We've fit the *noise* in the training data, which is what we need our model to ignore.

In the cell below, we fit a degree one polynomial. In other words, this is simple linear regression, and it's the right model for this circumstance. We also find the RMSE for the training and test sets, and plot the model along with the training and test data.

In [ ]:
# Fit simple model (low-degree polynomial regression)
low_degree=1
ldp = np.poly1d(np.polyfit(Xlin_train,Y_train,deg=low_degree))

# Get metrics
train_rmse = mean_squared_error(Y_train, ldp(Xlin_train), squared=False)
test_rmse = mean_squared_error(Y_test, ldp(Xlin_test), squared=False)

# Print
print('Training set RMSE: ',train_rmse)
print('Test set RMSE: ',test_rmse)

# Take a look
Xplot = np.linspace(-4,4)
fig, axs = plt.subplots(1,2,figsize=(12,5))
axs[0].plot(Xplot,ldp(Xplot))
axs[0].plot(Xlin_train,Y_train,'ro')
axs[0].plot(Xlin_test,Y_test,'bo')
axs[1].plot(Xplot,ldp(Xplot))
axs[1].plot(Xlin_train,Y_train,'ro')
axs[1].plot(Xlin_test,Y_test,'bo')
axs[1].set_ylim(-25,15)
plt.show()

Much better! Our training RMSE is still lower than our test RMSE, by a wide margin, but that's not unexpected, especially with such a small data set. They are much closer together than they were for the high-degree fit.

Now let's go all out -- let's fire up a neural network with a thousand internal nodes and apply it to our training data, to see the result.

In [ ]:
from sklearn.neural_network import MLPRegressor as Rgr

mlp_params = {
    'alpha':  np.logspace(-0.8,0.3,8, 30, 80),
    'hidden_layer_sizes': [(1000,)]
}

mlp_cv = GridSearchCV( Rgr(max_iter=400), mlp_params, scoring='neg_mean_squared_error', cv=4, n_jobs=6, verbose=0)
mlp_cv.fit(Xlin_train.reshape(-1, 1),Y_train)

In [ ]:
# Fit MLP neural network
mlpnn = mlp_cv.best_estimator_.predict

# Get metrics
train_rmse = mean_squared_error(Y_train, mlpnn(Xlin_train.reshape(-1, 1)), squared=False)
test_rmse = mean_squared_error(Y_test, mlpnn(Xlin_test.reshape(-1, 1)), squared=False)

# Print
print('Training set RMSE: ',train_rmse)
print('Test set RMSE: ',test_rmse)

# Take a look
Xplot = np.linspace(-4,4)
fig, axs = plt.subplots(1,2,figsize=(12,5))
axs[0].plot(Xplot,mlpnn(Xplot.reshape(-1, 1)))
axs[0].plot(Xlin_train,Y_train,'ro')
axs[0].plot(Xlin_test,Y_test,'bo')
axs[1].plot(Xplot,mlpnn(Xplot.reshape(-1, 1)))
axs[1].plot(Xlin_train,Y_train,'ro')
axs[1].plot(Xlin_test,Y_test,'bo')
axs[1].set_ylim(-25,15)
plt.show()

Not bad! This model too is overly complex, but it didn't do anywhere near as poorly as the high-degree polynomial fit. But it isn't as good as the simple linear model -- and given our "true" phenomenon, it never could be.

#### Reduce data complexity (dimensional reduction)

The second strategy we'll look at is reducing the complexity of your *data*, rather than that of your model. If you feed your model more data than it needs, it will use that data to "cheat" again. Think of it like this: even if some of the features you give your model are pure noise, completely useless, if you have enough of that data there will be some patterns in it (by pure chance) that the model can use to help itself memorize your training data and overfit.

In the two cells below we take the same small data set from above and complicate it by adding a bunch of useless features, that are pure noise. Then, using the same linear regression model that was best above, we'll see how the extra data complexity affects our fit.

In [ ]:
# Make overly complex data
Xcomplex = np.concatenate([Xlin.reshape(-1,1),
                           rng.normal(loc=0,scale=.5,size=datasize).reshape(-1,1),
                           rng.normal(loc=0,scale=.5,size=datasize).reshape(-1,1),
                           rng.normal(loc=0,scale=.5,size=datasize).reshape(-1,1),
                           rng.normal(loc=0,scale=.5,size=datasize).reshape(-1,1),
                           rng.normal(loc=0,scale=.5,size=datasize).reshape(-1,1),
                           rng.normal(loc=0,scale=.5,size=datasize).reshape(-1,1),
                           rng.normal(loc=0,scale=.5,size=datasize).reshape(-1,1),
                           rng.normal(loc=0,scale=.5,size=datasize).reshape(-1,1)],axis=1)

# Divide into train and test
Xcomplex_train, Xcomplex_test, Y_train, Y_test = train_test_split(
    Xcomplex, Y, test_size=0.5, random_state=355)


In [ ]:
# Fit simple model (linear regression)
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(Xcomplex_train,Y_train)

# Get metrics
train_rmse = mean_squared_error(Y_train, reg.predict(Xcomplex_train), squared=False)
test_rmse = mean_squared_error(Y_test, reg.predict(Xcomplex_test), squared=False)

# Print
print('Training set RMSE: ',train_rmse)
print('Test set RMSE: ',test_rmse)

Maybe this doesn't look so different from the good fit above, with Training RMSE around 3 and Test RMSE around 10. But look again at that Training RMSE from this fit -- it's almost 0. Our model is fitting itself almost *perfectly* to the training data, which makes it overfit.

#### Get more data

Let's look at our third strategy for preventing overfitting: get more data.

The bigger your training set, the less likely it is that your training data will look very different from the test set. Part of the reason it's so easy to overfit the example we've been looking at is that we've only got a handful of cases in our training data.

In the cell below, I generate a much bigger data set -- 500 cases -- from the same underlying phenomenon `Y = 3x + noise`. We'll see how this affects the fit of our simple linear regression.

In [ ]:
# Now try linear model with big data

# Make big data
datasize=500
Xlin_big = np.sort(rng.uniform(low=-4,high=4,size=datasize))
Y_big = 3*Xlin_big + rng.normal(loc=1,scale=8,size=datasize)

# Divide into train and test
Xlin_train, Xlin_test, Y_train, Y_test = train_test_split(
    Xlin_big, Y_big, test_size=0.5, random_state=355)

# Fit simple model (low-degree polynomial regression)
low_degree=1
ldp = np.poly1d(np.polyfit(Xlin_train,Y_train,deg=low_degree))

# Get metrics
train_rmse = mean_squared_error(Y_train, ldp(Xlin_train), squared=False)
test_rmse = mean_squared_error(Y_test, ldp(Xlin_test), squared=False)

# Print
print('Training set RMSE: ',train_rmse)
print('Test set RMSE: ',test_rmse)

# Take a look
Xplot = np.linspace(-4,4)
fig, axs = plt.subplots(1,2,figsize=(12,5))
axs[0].plot(Xplot,ldp(Xplot))
axs[0].plot(Xlin_train,Y_train,'ro')
axs[0].plot(Xlin_test,Y_test,'bo')
axs[1].plot(Xplot,ldp(Xplot))
axs[1].plot(Xlin_train,Y_train,'ro')
axs[1].plot(Xlin_test,Y_test,'bo')
axs[1].set_ylim(-25,15)
plt.show()

Now, here's a model that for sure is not overfitting. The training and test RMSE are very close. And notice that, with the bigger data, our training RMSE has gone UP! This should not concern us in the slightest. Our test RMSE has gone down -- that is what matters.

#### Regularize

#### Hyper-parameter search

##TODO:  Evaluating _Unsupervised_ Models